In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


# 경로 설정

In [ ]:
import os

In [ ]:
# 디렉토리 변경
os.chdir('drive/My Drive/Colab Notebooks/DACON/AIFrenz_Season2/')

# 데이터 불러오기

In [ ]:
import numpy as np

In [ ]:
train = np.load('./Data/dacon_npy/train.npy')

In [ ]:
x_train = train[:, :, :, :-1]
y_train = train[:, :, :, -1].reshape(-1, 40, 40, 1)

In [ ]:
del train

# 0보다 작은 값이 포함된 데이터 삭제

In [ ]:
y_train_ = y_train.reshape(-1, y_train.shape[1] * y_train.shape[2])

x_train = np.delete(x_train, np.where(y_train_ < 0)[0], axis=0)
y_train = np.delete(y_train, np.where(y_train_ < 0)[0], axis=0)

y_train = y_train.reshape(-1, x_train.shape[1], x_train.shape[2], 1)

y_train_ = np.delete(y_train_, np.where(y_train_ < 0)[0], axis=0)

x_train.shape, y_train.shape

((75957, 40, 40, 14), (75957, 40, 40, 1))

# 비가 내린 셀이 50개 이상인 데이터만 선택
- 모든 데이터를 학습에 사용하는 것과 결과에 큰 차이는 없지만 모든 데이터 사용시 학습 데이터의 양의 차이때문에 학습시간 차이가 심함

In [ ]:
x_train = x_train[np.where(np.sum(y_train_,axis=1) > 50)]
y_train = y_train[np.where(np.sum(y_train_,axis=1) > 50)]

x_train.shape, y_train.shape

((31406, 40, 40, 14), (31406, 40, 40, 1))

In [ ]:
del y_train_

# 평가 함수 정의
- 규칙에 정의된 함수

In [ ]:
from sklearn.metrics import f1_score

In [ ]:
def mae_over_fscore(y_true, y_pred):
    '''
    y_true: sample_submission.csv 형태의 실제 값
    y_pred: sample_submission.csv 형태의 예측 값
    '''

    y_true = np.array(y_true)
    y_true = y_true.reshape(1, -1)[0]  
    
    y_pred = np.array(y_pred)
    y_pred = y_pred.reshape(1, -1)[0]
    
    # 실제값이 0.1 이상인 픽셀의 위치 확인
    IsGreaterThanEqualTo_PointOne = y_true >= 0.1
    
    # 실제 값에 결측값이 없는 픽셀의 위치 확인 
    IsNotMissing = y_true >= 0
    
    # mae 계산
    mae = np.mean(np.abs(y_true[IsGreaterThanEqualTo_PointOne] - y_pred[IsGreaterThanEqualTo_PointOne]))
    
    # f1_score 계산 위해, 실제값에 결측값이 없는 픽셀에 대해 1과 0으로 값 변환
    y_true = np.where(y_true[IsNotMissing] >= 0.1, 1, 0)
    
    y_pred = np.where(y_pred[IsNotMissing] >= 0.1, 1, 0)
    
    # f1_score 계산    
    f_score = f1_score(y_true, y_pred)
    
    # f1_score가 0일 나올 경우를 대비하여 소량의 값 (1e-07) 추가 
    return mae / (f_score + 1e-07) 

# 손실 함수 정의
- 베이스라인에 정의된 함수

In [ ]:
import tensorflow as tf

In [ ]:
def mae(y_true, y_pred):    
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    
    y_true = y_true.reshape(1, -1)[0]
    
    y_pred = y_pred.reshape(1, -1)[0]
    
    over_threshold = y_true >= 0.1
    
    return np.mean(np.abs(y_true[over_threshold] - y_pred[over_threshold]))

def fscore(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    
    y_true = y_true.reshape(1, -1)[0]
    
    y_pred = y_pred.reshape(1, -1)[0]
    
    remove_NAs = y_true >= 0
    
    y_true = np.where(y_true[remove_NAs] >= 0.1, 1, 0)
    
    y_pred = np.where(y_pred[remove_NAs] >= 0.1, 1, 0)
    
    return(f1_score(y_true, y_pred))

def maeOverFscore(y_true, y_pred):
    return mae(y_true, y_pred) / (fscore(y_true, y_pred) + 1e-07)

def fscore_keras(y_true, y_pred):
    score = tf.py_function(func=fscore, inp=[y_true, y_pred], Tout=tf.float32, name='fscore_keras')
    return score

def score(y_true, y_pred):
    score = tf.py_function(func=maeOverFscore, inp=[y_true, y_pred], Tout=tf.float32,  name='custom_mse') 
    return score

# 모델 설계 및 학습
- 강수량을 예측할 지점에서 멀리 있는 데이터일 수록 영향이 적어져 풀링레이어가 결과 예측에 방해가 될 것으로 판단

- convolution과 residual block만으로 모델 설계

In [ ]:
def create_model():
    inputs = tf.keras.layers.Input(x_train.shape[1:])
    
    bn = tf.keras.layers.BatchNormalization()(inputs)
    conv0 = tf.keras.layers.Conv2D(256, kernel_size=1, strides=1, padding='same', activation='relu')(bn)
    
    bn = tf.keras.layers.BatchNormalization()(conv0)
    conv = tf.keras.layers.Conv2D(128, kernel_size=2, strides=1, padding='same', activation='relu')(bn)
    concat = tf.keras.layers.concatenate([conv0, conv], axis=3)
    
    bn = tf.keras.layers.BatchNormalization()(concat)
    conv = tf.keras.layers.Conv2D(64, kernel_size=3, strides=1, padding='same', activation='relu')(bn)
    concat = tf.keras.layers.concatenate([concat, conv], axis=3)
        
    for i in range(5):
        bn = tf.keras.layers.BatchNormalization()(concat)
        conv = tf.keras.layers.Conv2D(32, kernel_size=3, strides=1, padding='same', activation='relu')(bn)
        concat = tf.keras.layers.concatenate([concat, conv], axis=3)
    
    bn = tf.keras.layers.BatchNormalization()(concat)
    outputs = tf.keras.layers.Conv2D(1, kernel_size=1, strides=1, padding='same', activation='relu')(bn)
    
    model = tf.keras.models.Model(inputs=inputs, outputs=outputs)
    
    return model

# 교차검증

In [ ]:
from sklearn.model_selection import KFold

In [ ]:
def train_model(x_data, y_data, k, s):
    models = []
    k_fold = KFold(n_splits=k, shuffle=True, random_state=7777)
    
    model_number = 0
    for train_idx, val_idx in k_fold.split(x_data):
        if model_number <= s:
            print('model:', model_number)
            x_train, y_train = x_data[train_idx], y_data[train_idx]
            x_val, y_val = x_data[val_idx], y_data[val_idx]

            model = create_model()

            model.compile(loss='mae', optimizer='adam', metrics=[score, fscore_keras])

            callbacks_list = [
                tf.keras.callbacks.EarlyStopping(
                    monitor='val_score',
                    patience=3,
                    restore_best_weights=True
                ),
                tf.keras.callbacks.ReduceLROnPlateau(
                    monitor='val_score',
                    patience=3,
                    factor=0.8
                ),
                tf.keras.callbacks.ModelCheckpoint(
                    filepath='Model/ResNet/ckpt/model' + str(model_number)+'.h5',
                    monitor='val_score',
                    save_best_only=True
                )
            ]

            model.fit(x_train, y_train, epochs=50, batch_size=128, validation_data=(x_val, y_val), callbacks=callbacks_list)
            models.append(model)
            del model
        
        model_number += 1
    return models

In [ ]:
k = 5
models = train_model(x_train, y_train, k=k, s=4)

model: 0
Epoch 1/50
197/197 [==============================] - 254s 1s/step - loss: 0.2839 - score: 2.5625 - fscore_keras: 0.6799 - val_loss: 0.3393 - val_score: 28.7553 - val_fscore_keras: 0.1008 - lr: 0.0010
Epoch 2/50
197/197 [==============================] - 252s 1s/step - loss: 0.2386 - score: 2.0782 - fscore_keras: 0.7257 - val_loss: 0.2931 - val_score: 3.9042 - val_fscore_keras: 0.4916 - lr: 0.0010
Epoch 3/50
197/197 [==============================] - 253s 1s/step - loss: 0.2324 - score: 2.0024 - fscore_keras: 0.7341 - val_loss: 0.2387 - val_score: 2.2611 - val_fscore_keras: 0.6769 - lr: 0.0010
Epoch 4/50
197/197 [==============================] - 248s 1s/step - loss: 0.2267 - score: 1.9149 - fscore_keras: 0.7477 - val_loss: 0.2314 - val_score: 2.0380 - val_fscore_keras: 0.7185 - lr: 0.0010
Epoch 5/50
197/197 [==============================] - 246s 1s/step - loss: 0.2241 - score: 1.8791 - fscore_keras: 0.7527 - val_loss: 0.2267 - val_score: 1.9959 - val_fscore_keras: 0.7207 - l

In [ ]:
del x_train, y_train

# 가채점

In [ ]:
x_test = np.load('./Data/dacon_npy/test.npy')

In [ ]:
preds = []
for model in models:
    preds.append(model.predict(x_test))
    print(mae_over_fscore(y_test, preds[-1]))

In [ ]:
del x_test

In [ ]:
pred = np.array(preds).mean(axis=0)
print(mae_over_fscore(y_test, pred))

In [ ]:
del preds

# Submission

In [ ]:
submission = pd.read_csv('./Data/sample_submission.csv')

In [ ]:
submission.iloc[:,1:] = pred.reshape(-1,1600)
submission

In [ ]:
del pred

In [ ]:
submission.to_csv('submission.csv', index=False)